Run this file after running "1_Clean_Eora.ipynb"

This file is used to generate paratmers used for estimating trade cost and running models

# Set up

In [1]:
import os
import pandas as pd 
import numpy as np


In [3]:
# Set working directory
wd = os.path.expanduser("~/Dropbox/Tariff_Project_2017")
os.chdir(wd)
print(f"Current working directory: {os.getcwd()}")

# 3. Define data paths and parameters
# Path to raw Eora BP data
data_path = os.path.join(wd, "3_Result/eora_clean")


save_path = os.path.join(wd, "3_Result/parameters")


# Verify paths
print(f"Cleaned Eora data folder: {data_path}")

Current working directory: /Users/yaolangzhong/Dropbox/Tariff_Project_2017
Cleaned Eora data folder: /Users/yaolangzhong/Dropbox/Tariff_Project_2017/3_Result/eora_clean


# Part I ： Run this part after running "1_Clean_Eora.ipynb"

In [4]:
# ----------------------------
#  Read country / sector lists
# ----------------------------
country_list = (
    pd.read_csv(os.path.join(data_path, "country_list.csv"), header=0)["0"]
    .astype(str)
    .tolist()
)

sector_list = (
    pd.read_csv(os.path.join(data_path, "sector_list.csv"), header=0)["0"]
    .astype(str)
    .tolist()
)

label_list = pd.read_csv(os.path.join(data_path, "labels.csv"), header=0)

# ----------------------------
#  Basic dimension information
# ----------------------------
N = len(country_list)  # number of countries
J = len(sector_list)   # number of sectors

print(f"{N = }, {J = }")


N = 37, J = 23


In [5]:
# -------------------------------------------------
#  Read input‑output tables, final demand, value added
# -------------------------------------------------

io = (
    pd.read_csv(os.path.join(data_path, "T_final_2017.csv"), header=0)
    .fillna(0)           
    .to_numpy(float)      
)

fd = (
    pd.read_csv(os.path.join(data_path, "FD_final_2017.csv"), header=0)
    .fillna(0)
    .to_numpy(float)
)



va = (
    pd.read_csv(os.path.join(data_path, "VA_final_2017.csv"), header=0)
    .fillna(0)
    .to_numpy(float)
)

print(f"io shape: {io.shape}, fd shape: {fd.shape}, va shape: {va.shape}")

io shape: (851, 851), fd shape: (851, 222), va shape: (6, 851)


## alpha(N,J): Calibrate $\alpha_n^{j}$ (preference paramter) in household utility function 


In [6]:
# Final demand accounts include: 
# 1.Household final consumption 
# 2.Non-profit institutions serving households
# 3.Government final consumption 
# 4.Gross fixed capital formation
# 5.Changes in inventories 
# 6.Acquisitions less disposals of valuables

# Here I exclude 5 and 6 to better represent household preference
fd_n = 6 
fd_n_2 = 4
keep_fd_n = [0, 1, 2, 3]  

# Step 1: select the four accounts (exclude the last two)
cols_to_select = []
for c_idx in range(N):
    base = c_idx * fd_n
    cols_to_select.extend([base + a for a in keep_fd_n])

fd_select = fd[:, cols_to_select]      # (N*J, 4*N)

# Step 2：sum up over accounts for final demand 
fd_sum = fd_select.reshape(fd_select.shape[0], N, fd_n_2).sum(axis=2) # reshape to (NJ, N, 4 and sum up over the last dim)

# Step 3: sum up over countries for the same sector
fd_sum_reshaped = fd_sum.reshape(N, J, N)
fd_final = fd_sum_reshaped.sum(axis=0)         # (J, N)：（j,n）is country n's total comsumption in sector j, no matter where it comes from 

# ----------- correct above - checked by hand --------------

# Step 4: calculate shares
total_fd = fd_final.sum(axis=0, keepdims=True) # (1, N) # sum up over all sectors for each country, that is, each country's total final demand (total final consumption)
Alpha_jn = fd_final / total_fd                 # (J, N) 
Alpha_nj = Alpha_jn.T                          # (N, J)

# Step 5: adjust
Alpha_nj = np.clip(Alpha_nj, 0.00001, 0.99999)
Alpha_nj = Alpha_nj / Alpha_nj.sum(axis=1, keepdims=True)


In [7]:
# Check if it satisfies the model

alpha = Alpha_nj.copy()  # alpha is the final variable name we use in model simulation

# Check
# Check 1: For each n, the sum over j equals 1
sum_by_n = np.sum(alpha, axis=1)
check_sum = np.allclose(sum_by_n, 1, atol=1e-6)  # Allowing a small numerical tolerance
if check_sum:
    print("For each country, the sum over secotrs equals 1 ✅")
else:
    print("There are some country where the sum over sectors is not 1 ❌")
    print("The issue occurs at the following n indices:", np.where(np.abs(sum_by_n - 1) > 1e-6)[0])
    print("The corresponding sums are:", sum_by_n[np.where(np.abs(sum_by_n - 1) > 1e-6)[0]])
# Check 2: Every value in alpha is between 0 and 1
check_range = np.all((alpha >= 0) & (alpha <= 1))
if check_range:
    print("Every value in alpha is between 0 and 1 ✅")
else:
    print("There are values in alpha that are not between 0 and 1 ❌")
    print("These values are at positions:", np.where((alpha < 0) | (alpha > 1)))
    print("The values are:", alpha[(alpha < 0) | (alpha > 1)])



For each country, the sum over secotrs equals 1 ✅
Every value in alpha is between 0 and 1 ✅


In [8]:
# Save to csv, for check
alpha_df = pd.DataFrame(Alpha_nj,
                        index=country_list,
                        columns=sector_list)


#os.makedirs(save_path, exist_ok=True) 
alpha_df.to_csv(os.path.join(save_path, "Alpha_nj_2017.csv"))

print("Alpha_nj_2017.csv saved successfully.")

Alpha_nj_2017.csv saved successfully.


## GO(N,J): Calculate Gross Output of Intermediate Goods, Final Goods, and All Goods


In [9]:
# Total output: sum rows
io_GO = io.sum(axis=1)         # (NJ,1) # intermediate output: sum up for each row
fd_GO = fd_select.sum(axis=1)  # (NJ,1) # final output:sum up for each row. Previously, I incorrectely wrote fd_GO = fd.sum(axis=1)  
#fd_GO = fd.sum(axis=1)  # (NJ,1) # final output:sum up for each row. Previously, I incorrectely wrote fd_GO = fd.sum(axis=1)
GO_temp    = io_GO + fd_GO         # (NJ,)

GO = GO_temp.reshape(N, J, order="C")  # (N,J) # row - country, col - sector

go_file = os.path.join(save_path, "GO_2017.csv")
pd.DataFrame(GO, index=country_list, columns=sector_list).to_csv(go_file)

## $\beta(N,J)$: Calibrate $\gamma_n^j$ (value-added share) in production function 

In [10]:
# Value added accounts include: 1.Compensation of employees; 2.Taxes on production; 4.Net operating surplus; 5. Net mixed income; 6. Consumption of fixed capital
# Value added accounts exclude: 3. Subsidies on production
# Reason: for some countries, Subsidies on production is large negative number, resulting in the sum of VA negative

va_total = np.delete(va, 2, axis=0).sum(axis=0)   # (N·J,)

Gamma_vec = va_total / GO_temp                        # (N·J,)
Gamma_nj  = Gamma_vec.reshape(N, J, order="C")   # # row - country, col - sector


gt_one = Gamma_nj > 1
if gt_one.any():
    print(f"There are {gt_one.sum()} values greater than 1. They have been replaced with 0.99.")
    Gamma_nj[gt_one] = 0.99
else:
    print("No values are greater than 1.")

lt_zero = Gamma_nj < 0
if lt_zero.any():
    print(f"There are {lt_zero.sum()} values less than 0. They have been replaced with 0.01.")
    Gamma_nj[lt_zero] = 0.01
else:
    print("No values are less than 0.")

# Since further adjustment may be needed, DO NOT save it now; save after calculating gamma_njk

There are 23 values greater than 1. They have been replaced with 0.99.
No values are less than 0.


## $\gamma(N,J,J)$: Calibarte $\gamma_n^{j,k}$

Note: gamma(n,j,k) refers to the share of input from sector k in the production of nj 
NOTE: the last dimension (k) is from sector in this code, j dimension is destination sector in this code

In [44]:
# Step 1: sum up over sectors
# Sum up each sectors. eg. sector 1 = sum of sector 1 of each countries (row1, row1+J, row1+2J...)
# The aim is to sum up all goods from the same sector across countries (no matter where they from)

io_sector_sum = io.reshape(N, J, -1).sum(axis=0)      # (J, N*J) # first reshape to (N,J,NJ),then, sum up over N, that is ,sum up across sectors no matter where they comes from 

# Step 2: Calculate Total Input (column sums)
total_input = io.sum(axis=0)                          # (N*J,)

# Step 3: Gamma_k_nj: row are the input sectors; colums are user "country-sector"
with np.errstate(divide="ignore", invalid="ignore"):
    Gamma_k_nj = io_sector_sum / total_input          # (J, N*J) 
Gamma_nj_k = Gamma_k_nj.T                             # (N*J, J) row - used by; col - from sector

# Step 4: Check if Gamma_nj_k contains any NA values
nan_mask = np.isnan(Gamma_nj_k)
if nan_mask.any():
    rows_with_nan = np.where(nan_mask.any(axis=1))[0]   
    n_idx = rows_with_nan // J
    j_idx = rows_with_nan %  J
    Gamma_nj[n_idx, j_idx] = 0.9999
    Gamma_nj_k[nan_mask] = 0.0001
    print("NA values have been handled.")
else:
    print("No NA values found in Gamma_nj_k.")

# NA appears because input of "KOR - Recycling", "ESP - Others" are all zeros across all coutries and sectors

# Step 5: Check and handle values less than 0 or greater than 1 in Gamma_nj_k
lt_zero = Gamma_nj_k < 0
if lt_zero.any():
    rows = np.where(lt_zero.any(axis=1))[0]
    n_idx = rows // J
    j_idx = rows %  J
    Gamma_nj_k[lt_zero] = 0.0001
    Gamma_nj[n_idx, j_idx] = 0.9999
    handled = True
else:
    handled = False

gt_one = Gamma_nj_k > 1
if gt_one.any():
    rows = np.where(gt_one.any(axis=1))[0]
    n_idx = rows // J
    j_idx = rows %  J
    Gamma_nj_k[gt_one] = 0.9999
    Gamma_nj[n_idx, j_idx] = 0.0001
    handled = True

print("Values less than 0 or greater than 1 have been handled." if handled
    else "No values less than 0 or greater than 1 found in Gamma_nj_k.")

# Step 6: Adjust to meet the model
row_sum = Gamma_nj_k.sum(axis=1)             # (N*J,)
row_rest = 1.0 - Gamma_nj.ravel(order="C")   # (N*J,)


scale = row_rest / row_sum  
Gamma_nj_k *= scale[:, None] # (NJ,J)



NA values have been handled.
No values less than 0 or greater than 1 found in Gamma_nj_k.


In [45]:
# Check 

beta = Gamma_nj.copy()  # beta is the variable name used in model simulation

# Check: Every value in beta is between 0 and 1
check_range = np.all((beta >= 0) & (beta <= 1))
if check_range:
    print("Every value in beta is between 0 and 1 ✅")
else:
    print("There are values in beta that are not between 0 and 1 ❌")
    print("These values are at positions:", np.where((beta < 0) | (beta > 1)))
    print("The values are:", beta[(beta < 0) | (beta > 1)])



Every value in beta is between 0 and 1 ✅


In [46]:
# Check 
# Check 1: Every value in gamma is between 0 and 1

gamma = Gamma_nj_k.reshape((N, J, J)) # gamma is the variable name used in model simulation:  $gamma(n, j, k)$ : country n, using sector j, producing sector k

invalid_values = (gamma < 0) | (gamma > 1)

if np.any(invalid_values):
    print("There are values in gamma that are not between 0 and 1 ❌")
    print("These values are at positions:", np.where(invalid_values))
    print("The values are:", gamma[invalid_values])
else:
    print("Every value in gamma is between 0 and 1 ✅")

# Check 2: sum(k) gamma[n,j,k] + beta[n,j] = 1
temp = np.sum(gamma, axis = 2) + beta
is_valid = np.allclose(temp, 1, atol = 1e-5)

if is_valid:
    print("Condition satisfied: sum(k) gamma[n, j, k] + beta[n, j] = 1 ✅")
else:
    print("Condition not satisfied: sum(k) gamma[n, k, j] + beta[n, j] ≠ 1 ❌")
    print("Positions where the condition fails:", np.where(~np.isclose(temp, 1, atol=1e-5)))
    print("Values that do not satisfy the condition:", temp[~np.isclose(temp, 1, atol=1e-5)])

Every value in gamma is between 0 and 1 ✅
Condition satisfied: sum(k) gamma[n, j, k] + beta[n, j] = 1 ✅


In [47]:
# Save to csv
gamma_njk_df = pd.DataFrame(Gamma_nj_k,
                            index=[f"{country}_{sector}"
                                for country in country_list
                                for sector  in sector_list],
                            columns=sector_list)

gamma_nj_df  = pd.DataFrame(Gamma_nj,
                            index=country_list,
                            columns=sector_list)

gamma_njk_df.to_csv(os.path.join(save_path, "Gamma_njk_2017.csv"))
gamma_nj_df.to_csv (os.path.join(save_path, "Gamma_nj_2017.csv"))

print("Gamma_njk_2017.csv and Gamma_nj_2017.csv saved successfully.")

Gamma_njk_2017.csv and Gamma_nj_2017.csv saved successfully.


## VA: shape (N, )

In [48]:
va_matrix = va_total.reshape(N, J)   

VA = va_matrix.sum(axis=1)

df_va = pd.DataFrame({
    "Country": country_list,
    "ValueAdded": VA
})

df_va.to_csv("3_Result/parameters/VA.csv", index=False, encoding="utf-8")


# Check if every value in VA is greater than 0
check_positive = np.all(VA > 0)
if check_positive:
    print("Every country's value added is greater than 0 ✅")
else:
    print("There are values in VA that are less than or equal to 0 ❌")
    print("These values are at positions:", np.where(VA <= 0))
    print("The values are:", VA[VA <= 0])

Every country's value added is greater than 0 ✅


# Part II: Run this part after running "3_Estimate_TradeCost.ipynb"


## $\theta_n$

As that decided in "3_Estimate_TradeCost.ipynb"

In [49]:
# Trade elasticity: take from "Bolhuis, M. A., Chen, M. J., & Kett, B. R. (2023). Fragmentation in global trade: Accounting for commodities. International Monetary Fund."

##############  Sector megering case ######################

# 1 Agriculture, Fishing: 2.91                               
# 2 Mining and Quarrying: 3.41                               
# 3 Food & Beverages: 4.17                                     
# 4 Textiles and Wearing Apparel: 4.71
# 5 Wood and Paper: (8.8 + 8.21) /2 = 8.505                                                                    
# 6 Petroleum, Chemical and Non-Metallic Mineral Products:  (3.67 + 10.56 + 6.75 + 4.79)/4 = 6.4425
# 7 Metal Products: （7.39 + 4.22）/2 = 5.805
# 8 Electrical and Machinery:  (5.01 + 5.14 + 4.11) /3 = 4.753                                                      
# 9 Transport Equipment: (8.92 + 8.99)/2 = 8.955                               
# 10 Other Manufacturing, Recycling :  4.06                
# ---------------------------------------------------------------------
# Services sectors: 8.35

theta = np.array([2.91, 3.14, 4.17, 4.71, 8.505, 6.4425, 5.805, 4.753, 8.955, 4.06,
                8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35])



## $\tilde{\tau}$: shape $(N, N, J)$
The tariff rate vector, the $(n, i, j)$ denote the $(1+\tau)$ tariff rate of country $n$ on country $i$ for goods of sector $j$

In [50]:
# Import tariff data constructed in "3_Estimate_TradeCost.ipynb"
# Here I choose "tariff" column to apply to my model

tariff_all  = pd.read_csv(os.path.join(save_path, "All_Tariff_2017.csv"))
tariff_df  = tariff_all .iloc[:, 1:]
tariff_df = tariff_df[['Importer', 'Exporter', 'Sector', "tariff"]]

tariff_df['Exporter_Sector'] = tariff_df['Exporter'] + '_' + tariff_df['Sector']
tariff_df['Importer'] = pd.Categorical(
    tariff_df['Importer'],
    categories=tariff_df['Importer'].unique(),
    ordered=True
)
tariff_df['Exporter_Sector'] = pd.Categorical(
    tariff_df['Exporter_Sector'],
    categories=tariff_df['Exporter_Sector'].unique(),
    ordered=True
)

# pivot and reshape
tariff_matrix = tariff_df.pivot_table(
    index='Importer',
    columns='Exporter_Sector',
    values='tariff',
    aggfunc='first',
    observed=False
)
tariff_np   = tariff_matrix.to_numpy() / 100
tariff_base = tariff_np.reshape((N, N, J))

# Construct tilde_tau
# tilde_tau[n,i,j]: country n's tariff on goods j from contry i (n:importer, i:exporte)
tilde_tau = tariff_base + 1


In [51]:
tariff_np

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.68311932e-02, 1.14054054e-04, 2.84955205e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.68311932e-02, 1.14054054e-04, 2.75753640e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.60273172e-03, 7.83783800e-05, 5.86006451e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.33963087e-02, 3.19669670e-03, 3.54108935e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.91954600e-02, 4.30856486e-02, 3.56773959e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

## Xf(N,J) and Xm(N,J): Calculate Expenditure for final goods and intermediate goods

This is not necessary for model simulation; just for check

In [52]:
# The observed data is basic price

# To obatin X in purchase price, we multiply by tau (1 + tariff)

Mm_temp = io.reshape(N, J, N, J).sum(axis = 3) # (exporter, sector, importer)
Mm = Mm_temp.transpose(2, 0, 1)  # (importer, exporter, sector) 
# adjust by \tau
tau_esI = np.transpose(tilde_tau, (1, 2, 0))   

Xm_bilateral = Mm_temp * tau_esI # (exporter, sector, importer)
Xm  =  Xm_bilateral.sum(axis = 0 ).T   # (N,J)


Mf_temp = fd_select.reshape(N,J,N,fd_n_2).sum(axis = 3) # (exporter, sector, importer)
Mf = Mf_temp.transpose(2, 0, 1)  # (importer, exporter, sector)
Xf_bilateral = Mf_temp  * tau_esI # (exporter, sector, importer)
Xf  =  Xf_bilateral.sum(axis = 0 ).T # (N,J)  

M = Mm + Mf   # (importer, exporter, sector) 

X = Xf + Xm  # (N,J)


# save
df_X = pd.DataFrame(X,
                index=country_list,
                columns=sector_list)
df_X.to_csv("3_Result/parameters/X.csv", index=False, encoding="utf-8")

df_Xm = pd.DataFrame(Xm,
                index=country_list,
                columns=sector_list)
df_Xm.to_csv("3_Result/parameters/Xm.csv", index=False, encoding="utf-8")


df_Xf = pd.DataFrame(Xf,
                index=country_list,
                columns=sector_list)
df_Xf.to_csv("3_Result/parameters/Xf.csv", index=False, encoding="utf-8")



##  $D_n$: Trade Deficit (Import - Export)

In [53]:
# M: shape (N, N, J)
imports = M.sum(axis=1)   # shape (N, J) ─ Σ_i M_{n i}^j
exports = M.sum(axis=0)   # shape (N, J) ─ Σ_i M_{i n}^j

D_nj = imports - exports     # shape (N, J) ─ D_n^j

D = D_nj.sum(axis=1) # D.shape (N,)

df = pd.DataFrame({
    'Country': country_list,
    'IM  - EX': D
})

df.to_csv("3_Result/parameters/D.csv", index=False, encoding="utf-8")


## pif,pim,pit(N,N,J): Calibrate intermediate goods trade share $\pi_{ni}^{j,I}$, final goods trade share $\pi_{ni}^{j,F}$, and total trade share $\pi_{ni}^{j}$

pi(n,i,j) n is  importer, i is exporter, j is sector

In [54]:
Xf_nij= Xf_bilateral.transpose(2, 0, 1)  # (importer, exporter, sector)
Xm_nij = Xm_bilateral.transpose(2, 0, 1)  # (importer, exporter, sector)
X_nij = Xf_nij + Xm_nij  # (importer, exporter, sector)


# Total trade share
denom = X_nij.sum(axis=1, keepdims=True)        # shape (N, 1, J)
with np.errstate(divide="ignore", invalid="ignore"):
    pi = np.divide(X_nij, denom, where=denom != 0)   
    pi = np.nan_to_num(pi)

# Intermediate goods trade share
denom = Xm_nij.sum(axis=1, keepdims=True)        # shape (N, 1, J)
with np.errstate(divide="ignore", invalid="ignore"):
    pim = np.divide(Xm_nij, denom, where=denom != 0)   
    pim = np.nan_to_num(pim)


# Final goods trade share
denom = Xf_nij.sum(axis=1, keepdims=True)        # shape (N, 1, J)
with np.errstate(divide="ignore", invalid="ignore"):
    pif = np.divide(Xf_nij, denom, where=denom != 0)   
    pif = np.nan_to_num(pif)

# Check

pi_sets = {
    "final": pif,
    "intermediate": pim,
    "all": pi
}

tol_sum = 1e-5          
tol_neg = -1e-12       

for name, arr in pi_sets.items():
    # ---- 1)check negative ----
    neg_mask = arr < tol_neg         
    if np.any(neg_mask):
        coords = np.argwhere(neg_mask)
        print(f"{name:>12} ❌ Found {coords.shape[0]} negative entries")
        for idx in coords[:10]:
            n, i, j = idx
            print(f"  (importer={n}, exporter={i}, sector={j}) -> {arr[n, i, j]}")
        if coords.shape[0] > 10:
            print("  ... (truncated)")
    else:
        print(f"{name:>12} ✅ No negative entries\n")
    # ---- 2) check sum ----
    col_sum = np.sum(arr, axis=1)
    if np.allclose(col_sum, 1, atol=tol_sum):
        print(f"{name:>12} ✅ Condition satisfied: ∑₍i₎ π[{name}][n,i,j] = 1")
    else:
        bad = ~np.isclose(col_sum, 1, atol=tol_sum)
        print(f"{name:>12} ❌ Condition not satisfied: {bad.sum()} (n,j) positions differ")
        print("  Position (n,j):", np.argwhere(bad))
        print("  Actual sum:", col_sum[bad])



       final ✅ No negative entries

       final ✅ Condition satisfied: ∑₍i₎ π[final][n,i,j] = 1
intermediate ✅ No negative entries

intermediate ✅ Condition satisfied: ∑₍i₎ π[intermediate][n,i,j] = 1
         all ✅ No negative entries

         all ✅ Condition satisfied: ∑₍i₎ π[all][n,i,j] = 1


In [59]:
def country_reordering(country_list, ordering_var, ordering_type="desc"):
    # Step 1: Get sorted indices (descending by ordering_var)
    if ordering_type == "desc":
        sorted_indices = np.argsort(-ordering_var).astype(int)
    elif ordering_type == "asc":
        sorted_indices = np.argsort(ordering_var).astype(int)
    else:
        raise ValueError(f"Invalid ordering type: {ordering_type}")
    
    # Step 2: Find ROW index in the sorted list
    sorted_countries = country_list[sorted_indices]
    row_idx_in_sorted = np.where(sorted_countries == "ROW")[0]
    
    if len(row_idx_in_sorted) == 0:
        # If ROW not found, just use the sorted order
        final_indices = sorted_indices
    else:
        row_idx_in_sorted = row_idx_in_sorted[0]  # Get the index
        
        # Step 3: Create final ordering (move ROW to end)
        final_indices = np.concatenate([
            sorted_indices[:row_idx_in_sorted],      # Before ROW
            sorted_indices[row_idx_in_sorted+1:],    # After ROW
            sorted_indices[row_idx_in_sorted:row_idx_in_sorted+1]  # ROW at end
        ])
    
    N = len(country_list)
    
    # Step 4: Create optimized reordering function
    def reorder_variable(variable):
        """
        Efficiently reorder variable along all dimensions matching country size N
        
        Args:
            variable: Input array of any shape where dimensions of size N 
                     correspond to countries in original country_list order
            
        Returns:
            Reordered array with same shape, country dimensions reordered
        """
        variable = np.asarray(variable)
        
        # Find country dimensions (size N)
        country_dims = np.where(np.array(variable.shape) == N)[0]
        
        if len(country_dims) == 0:
            return variable
        
        # Apply reordering to all country dimensions
        result = variable
        for dim in country_dims:
            result = result.take(final_indices, axis=dim)
        
        return result
    
    return reorder_variable, sorted_countries

In [ ]:
reorder_variable, sorted_countries = country_reordering(np.asarray(country_list), VA)

In [63]:
country_list = reorder_variable(country_list)
sector_list = reorder_variable(sector_list)
alpha = reorder_variable(alpha)
beta = reorder_variable(beta)
gamma = reorder_variable(gamma)
theta = reorder_variable(theta)
pi = reorder_variable(pi)
pim = reorder_variable(pim)
pif = reorder_variable(pif)
Xf = reorder_variable(Xf)
Xm = reorder_variable(Xm)
X = reorder_variable(X)
tilde_tau = reorder_variable(tilde_tau)
D = reorder_variable(D)
VA = reorder_variable(VA)

In [71]:
os.chdir("/Users/yaolangzhong/Research/volatility_analysis")

In [74]:
#np.savez('data_2017(gvc_consistent).npz',
np.savez('data.npz',
        N = N,
        S = J,
        country_list = country_list, 
        sector_list = sector_list, 
        alpha = alpha, 
        beta = beta, 
        gamma = gamma.transpose(0, 2, 1),
        theta = theta, 
        pi = pi,
        pim = pim,
        pif = pif,
        Xf = Xf,
        Xm = Xm,
        X = X,
        tilde_tau = tilde_tau,
        D = D, 
        V = VA)

In [26]:
real_data = np.load('data_2017(gvc_consistent).npz')
N = int(real_data['N'])
J = int(real_data['S'])
country_list = [c.decode("utf-8") if isinstance(c, bytes) else c
                for c in real_data["country_list"].tolist()]
sector_list = [s.decode("utf-8") if isinstance(s, bytes) else s
                for s in real_data["sector_list"].tolist()]
theta = real_data['theta']
X = real_data['X'] 
alpha = real_data['alpha']
beta = real_data['beta']
gamma = real_data['gamma']
pi = real_data['pi']

gamma_copy = gamma.copy()  
gamma_cp = gamma_copy.transpose(0, 2, 1)  

D_copy = D.copy()
D_cp = -D_copy



In [27]:
from equilibrium import equilibrium
from cp_functions import calc_real_I, calc_real_w

kappa_base = np.ones((N,N,J))

w_hat_initial = np.ones(N)
P_hat_initial = np.ones((N, J)) 

w_base, P_base, X_base, pi_base = equilibrium(gamma_cp, beta, theta, tilde_tau, kappa_base, pi, alpha, VA, D_cp, N, J, X, w_hat_initial, P_hat_initial)


Round 1: w_hat_min = 0.9839700254858132, w_hat_max = 1.0119169217906454, min_X_prime = 14487.984317812025, max_X_prime = 9684121197.426102, wfmax = 0.01602997451418675, Pfmax = 2.220446049250313e-16 
Round 2: w_hat_min = 0.9711986100711048, w_hat_max = 1.0216147564145657, min_X_prime = 14466.195222466045, max_X_prime = 9724650796.536306, wfmax = 0.012771415414708498, Pfmax = 0.0116643143030839 
Round 3: w_hat_min = 0.9626306099937414, w_hat_max = 1.0288040372013172, min_X_prime = 14444.054832388061, max_X_prime = 9754502189.540283, wfmax = 0.008568000077363358, Pfmax = 0.009555375298280211 
Round 4: w_hat_min = 0.9578287561028535, w_hat_max = 1.0339785923089162, min_X_prime = 14425.249829327668, max_X_prime = 9776798162.539291, wfmax = 0.00517455510759901, Pfmax = 0.00832994617231142 
Round 5: w_hat_min = 0.9557644563046049, w_hat_max = 1.0376981190617718, min_X_prime = 14411.612099082482, max_X_prime = 9793714080.868353, wfmax = 0.003719526752855584, Pfmax = 0.006190661081475923 
Roun

In [33]:
%load_ext autoreload
%autoreload 2  

from equilibrium_T import equilibrium
from cp_functions_T import calc_real_I, calc_real_w

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
kappa_base = np.ones((N,N,J))
T_base = np.ones((N,J))

w_hat_initial = np.ones(N)
P_hat_initial = np.ones((N, J)) 

w_base, P_base, X_base, pi_base = equilibrium(gamma_cp, beta, theta, tilde_tau, kappa_base, T_base, pi, alpha, VA, D_cp, N, J, X, w_hat_initial, P_hat_initial)


Round 1: w_hat_min = 0.9839700254858131, w_hat_max = 1.0119169217906452, min_X_prime = 14487.984317812028, max_X_prime = 9684121197.426104, wfmax = 0.016029974514186862, Pfmax = 2.220446049250313e-16 
Round 2: w_hat_min = 0.9711986100711048, w_hat_max = 1.0216147564145657, min_X_prime = 14466.195222466045, max_X_prime = 9724650796.536308, wfmax = 0.012771415414708387, Pfmax = 0.0116643143030839 
Round 3: w_hat_min = 0.9626306099937414, w_hat_max = 1.0288040372013172, min_X_prime = 14444.054832388065, max_X_prime = 9754502189.540285, wfmax = 0.008568000077363358, Pfmax = 0.0095553752982801 
Round 4: w_hat_min = 0.9578287561028536, w_hat_max = 1.0339785923089162, min_X_prime = 14425.249829327673, max_X_prime = 9776798162.539291, wfmax = 0.00517455510759901, Pfmax = 0.00832994617231142 
Round 5: w_hat_min = 0.9557644563046049, w_hat_max = 1.0376981190617718, min_X_prime = 14411.612099082486, max_X_prime = 9793714080.868351, wfmax = 0.003719526752855584, Pfmax = 0.006190661081475923 
Round

\begin{equation}
\hat{c}_n^j=\hat{w}_n^{\gamma_n^j} \prod_{k=1}^J (\hat{P}_n^k)^{\gamma_n^{k j}}
\end{equation}

\begin{equation}
\hat{P}_n^j=\left[\sum_{i=1}^N \pi_{n i}^j \hat{T}_i^j\left(\hat{c}_i^j \hat{\kappa}_{n i}^j\right)^{-\theta^j}\right]^{-\frac{1}{\theta^j}}
\end{equation}

\begin{equation}
\hat{\pi}_{n i}^j=\frac{\hat{T}_i^j\left(\hat{c}_i^j \hat{\kappa}_{n i}^j\right)^{-\theta^j}}{\sum_{h=1}^N \pi_{n h}^j \hat{T}_h^j\left(\hat{c}_h^j \hat{\kappa}_{n h}^j\right)^{-\theta^j}}
\end{equation}

\begin{equation}
X_n^{j^{\prime}}=\sum_{k=1}^J \gamma_n^{j, k} \sum_{i=1}^N \frac{\pi_{i n}^{k^{\prime}}}{1+\tau_{i n}^{k^{\prime}}} X_i^{k^{\prime}}+\alpha_n^j\left(\widehat{w}_n w_n L_n+\sum_{j=1}^J \sum_{i=1}^N \tau_{n i}^{j^{\prime}} \frac{\pi_{n i}^{j^{\prime}}}{1+\tau_{n i}^{j^{\prime}}} X_n^{j^{\prime}}+D_n\right)
\end{equation}


\begin{equation}
\sum_{j=1}^J \sum_{i=1}^N \frac{\pi_{n i}^{j^{\prime}}}{1+\tau_{n i}^{j^{\prime}}} X_n^{j^{\prime}}-D_n=\sum_{j=1}^J \sum_{i=1}^N \frac{\pi_{i n}^{j^{\prime}}}{1+\tau_{i n}^{j^{\prime}}} X_i^{j^{\prime}}
\end{equation}

# Check

In [ ]:
from equilibrium import equilibrium
from cp_functions import calc_real_I, calc_real_w

kappa_base = np.ones((N,N,J))

w_hat_initial = np.ones(N)
P_hat_initial = np.ones((N, J)) 

D_2 = -D

g_tem = gamma.copy()  # gamma is the variable name used in model simulation
gamma_TRY = g_tem.transpose(0, 2, 1)  

w_base, P_base, X_base, pi_base = equilibrium(gamma_TRY, beta, theta, tilde_tau, kappa_base, pi, alpha, VA, D_2, N, J, X, w_hat_initial, P_hat_initial)


In [ ]:
# real_data = np.load('model_data_2017.npz')
real_data = np.load('R_model_data_2017.npz')

for name in real_data.files:
    arr = real_data[name]
    print(f"{name}: shape={arr.shape}, dtype={arr.dtype}")



In [ ]:
N_R = int(real_data['N'])
J_R = int(real_data['J'])
country_list_R = [c.decode("utf-8") if isinstance(c, bytes) else c
                for c in real_data["country_list"].tolist()]
sector_list_R = [s.decode("utf-8") if isinstance(s, bytes) else s
                for s in real_data["sector_list"].tolist()]
theta_R = real_data['theta']
X_R = real_data['X']  # X[n,j]: country n's total output in sector j
alpha_R = real_data['alpha']
beta_R = real_data['beta']
gamma_R = real_data['gamma']
pi_R = real_data['pi']


In [ ]:


pairs_to_check = {
    "N":     (N,      N_R),
    "J":     (J,      J_R),
    "alpha": (alpha,  alpha_R),
    "beta":  (beta,   beta_R),
    "gamma": (gamma_TRY,  gamma_R),
    "theta": (theta,  theta_R),
}

ABS_TOL = 1e-3          # 仅按绝对误差判断，阈值 0.0001
total_mismatch = 0
report = []

for name, (lhs, rhs) in pairs_to_check.items():
    a, b = np.asarray(lhs), np.asarray(rhs)

    # 形状不同直接记为全不一致
    if a.shape != b.shape:
        report.append(f"[{name}] 形状不一致 {a.shape} vs {b.shape} → 计为全部不一致")
        mism = a.size
    else:
        diff = np.abs(a - b)
        mask_equal = diff <= ABS_TOL          # 绝对误差≤阈值视为一致
        mism = diff.size - mask_equal.sum()

        if mism == 0:
            report.append(f"[{name}] ✔︎ 全部一致")
        else:
            pos_max = np.unravel_index(diff.argmax(), a.shape)
            max_err = diff[pos_max]
            report.append(
                f"[{name}] ❌ 不一致 {mism} 个；最大误差 {max_err:.6g} @ {pos_max}"
            )

    total_mismatch += mism

print("\n".join(report))
print(f"\n=== 总计不一致元素数: {total_mismatch} ===")


In [ ]:
np.savez('data_2017(gvc_consistent).npz',
        N = N,
        J = J,
        country_list = country_list, 
        sector_list = sector_list, 
        alpha = alpha, 
        beta = beta, 
        gamma = gamma, 
        theta = theta, 
        pi = pi,
        X = X,
        tilde_tau = tilde_tau,
        D = D, 
        VA = VA)